# Lookback Lens 🔎 🦙

Demo code for the paper __"Detecting and Mitigating Contextual Hallucinations in Large Language Models Using Only Attention Maps"__


> TL;DR: We designed a method only using attention maps information (lookback ratio) to detect contextual hallucinations. The detector can be used to reduce hallucination during decoding.

Paper: https://arxiv.org/abs/2407.07071  
Code: https://github.com/voidism/Lookback-Lens  
Twitter discussion: https://x.com/YungSungChuang/status/1811047314397724882  

### Installation

In [2]:
!git clone https://github.com/voidism/Lookback-Lens.git

Cloning into 'Lookback-Lens'...
remote: Enumerating objects: 3809, done.
remote: Counting objects: 100% (3809/3809), done.
remote: Compressing objects: 100% (2766/2766), done.
remote: Total 3809 (delta 998), reused 3808 (delta 997), pack-reused 0
Receiving objects: 100% (3809/3809), 22.87 MiB | 16.46 MiB/s, done.
Resolving deltas: 100% (998/998), done.


In [4]:
!cd Lookback-Lens && pip install -r requirements.txt
!cd Lookback-Lens/transformers-4.32.0 && pip install -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!cd Lookback-Lens && gzip -d data/nq-open-10_total_documents_gold_at_4.jsonl.gz

## Skip Step 01 & 02 by downloading the precomputed lookback ratios & annotations.

In [7]:
!cd Lookback-Lens && wget "https://www.dropbox.com/scl/fi/a87iv6xw9xma6ppc5pw2h/step1and2.tar.bz?rlkey=j382rsrwu2wnfwj7sn14ai3qw&dl=0" -O step1and2.tar.bz
!cd Lookback-Lens && tar -xvf step1and2.tar.bz
!cd Lookback-Lens && mv dump/* . && rm -rf dump

--2024-07-25 05:10:46--  https://www.dropbox.com/scl/fi/a87iv6xw9xma6ppc5pw2h/step1and2.tar.bz?rlkey=j382rsrwu2wnfwj7sn14ai3qw&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc529bea9f92dd78483d24ccf483.dl.dropboxusercontent.com/cd/0/inline/CXVHNAdY4q2riyVmNovhSFkuH8fwaZcOvCDsd-G-Zykazi7ruDOBsVQhXL2lBzyEw1ubi859ZestX43eCc5FuvGSimZh5KBmxJ1N9zdfmdg6TT7u3jKpVnRgNlorVZkUZNHXIeNv1fVBInz6Ng-zmT62/file# [following]
--2024-07-25 05:10:46--  https://uc529bea9f92dd78483d24ccf483.dl.dropboxusercontent.com/cd/0/inline/CXVHNAdY4q2riyVmNovhSFkuH8fwaZcOvCDsd-G-Zykazi7ruDOBsVQhXL2lBzyEw1ubi859ZestX43eCc5FuvGSimZh5KBmxJ1N9zdfmdg6TT7u3jKpVnRgNlorVZkUZNHXIeNv1fVBInz6Ng-zmT62/file
Resolving uc529bea9f92dd78483d24ccf483.dl.dropboxusercontent.com (uc529bea9f92dd78483d24ccf483.dl.dropboxusercontent.com)... 162.125.65.

## Step 03: Fitting Lookback Lens Classifiers (NQ and CNN/DM)


> **Please replace `<hf_auth_token>` with your huggingface auth token for LLaMA2 access.**



### Predefined Span

In [12]:
!cd Lookback-Lens && python step03_lookback_lens.py --auth_token <hf_auth_token> --anno_1 anno-nq-7b.jsonl --anno_2 anno-cnndm-7b.jsonl --lookback_ratio_1 lookback-ratio-nq-7b.pt --lookback_ratio_2 lookback-ratio-cnndm-7b.pt

======== Loading data from anno-nq-7b.jsonl and lookback-ratio-nq-7b.pt...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100% 1.62k/1.62k [00:00<00:00, 6.45MB/s]
tokenizer.model: 100% 500k/500k [00:00<00:00, 15.0MB/s]
tokenizer.json: 100% 1.84M/1.84M [00:00<00:00, 13.5MB/s]
special_tokens_map.json: 100% 414/414 [00:00<00:00, 2.61MB/s]
2024-07-25 05:22:33.816180: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 05:22:33.816260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has alrea

### Sliding Window (=8)

In [13]:
!cd Lookback-Lens && python step03_lookback_lens.py --auth_token <hf_auth_token> --anno_1 anno-nq-7b.jsonl --anno_2 anno-cnndm-7b.jsonl --lookback_ratio_1 lookback-ratio-nq-7b.pt --lookback_ratio_2 lookback-ratio-cnndm-7b.pt --sliding_window 8

======== Loading data from anno-nq-7b.jsonl and lookback-ratio-nq-7b.pt...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-07-25 05:32:34.530184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 05:32:34.530253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 05:32:34.539030: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-0

## Step 04: Run Greedy vs Classifier Guided Decoding (NQ and XSum)

> **Please replace `<hf_auth_token>` with your huggingface auth token for LLaMA2 access.**

Warning: Colab Pro is required to run decoding, as inference with LLaMA has high-RAM demand. Choose V100 GPU and turn on the High-RAM Shape option before running the code!

Warning: Running the code without High-RAM Shape option, the program will fail during loading the LLaMA checkpoints!



### Greedy (NQ)

In [ ]:
! cd Lookback-Lens && python step04_run_decoding.py --auth_token <hf_auth_token> --model-name meta-llama/Llama-2-7b-chat-hf --data-path data/nq-open-10_total_documents_gold_at_4.jsonl --output-path output-nq-open-greedy-decoding.jsonl --num-gpus 1

### Lookback Lens Guided Decoding (NQ)

In [ ]:
! cd Lookback-Lens && python step04_run_decoding.py --auth_token <hf_auth_token> --model_name meta-llama/Llama-2-7b-chat-hf --data_path data/nq-open-10_total_documents_gold_at_4.jsonl --output_path output-nq-open-lookback-decoding.jsonl --num_gpus 1 --do_sample --guiding_classifier classifiers/classifier_anno-cnndm-7b_sliding_window_8.pkl --chunk_size 8 --num_candidates 8

### Greedy (XSum)

In [ ]:
! cd Lookback-Lens && python step04_run_decoding.py --auth_token <hf_auth_token> --model_name meta-llama/Llama-2-7b-chat-hf --data_path data/xsum-1000.jsonl --output_path output-xsum-greedy-decoding.jsonl --num_gpus 1

### Lookback Lens Guided Decoding (XSum)

In [ ]:
! cd Lookback-Lens && python step04_run_decoding.py --auth_token <hf_auth_token> --model_name meta-llama/Llama-2-7b-chat-hf --data_path data/xsum-1000.jsonl --output_path output-xsum-lookback-decoding.jsonl --num_gpus 1 --do_sample --guiding_classifier classifiers/classifier_anno-cnndm-7b_sliding_window_8.pkl --chunk_size 8 --num_candidates 8

## Step 05: Run Evaluation

### Run Exact Match Evaluation (NQ)

In [ ]:
python eval_exact_match.py --hyp output-nq-open-greedy-decoding.jsonl --ref data/nq-open-10_total_documents_gold_at_4.jsonl
python eval_exact_match.py --hyp output-nq-open-lookback-decoding.jsonl --ref data/nq-open-10_total_documents_gold_at_4.jsonl

### Run GPT-4o Evaluation (XSum)

> Please set OpenAI API Key by `OPENAI_API_KEY={your_key}`.

In [ ]:
OPENAI_API_KEY={your_key} python step02_eval_gpt4o.py --hyp output-xsum-greedy-decoding.jsonl --ref data/xsum-1000.jsonl --out record-gpt4o-eval-xsum-greedy-decoding.jsonl
OPENAI_API_KEY={your_key} python step02_eval_gpt4o.py --hyp output-xsum-lookback-decoding.jsonl --ref data/xsum-1000.jsonl --out record-gpt4o-eval-xsum-lookback-decoding.jsonl